<a href="https://colab.research.google.com/github/MyShroomBot/MyShroomBot/blob/main/RESNET_Myshroom.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import torch
import torch.nn as nn
import torchvision
import torchvision.models as models
import torchvision.transforms as transforms
import torch.optim as optim
from torch.utils.data import DataLoader
from torchvision import datasets, models, transforms
import matplotlib.pyplot as plt
import numpy as np
import os
from torch.optim import lr_scheduler
import torch.backends.cudnn as cudnn
from tqdm import tqdm
import zipfile
cudnn.benchmark = True

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.optim import lr_scheduler
import torch.backends.cudnn as cudnn
import numpy as np
import torchvision
from torchvision import datasets, models, transforms
import matplotlib.pyplot as plt
import time
import os
from PIL import Image
from tempfile import TemporaryDirectory

In [ ]:
import numpy as np
import cv2
import io
from torch.utils.data import DataLoader, Dataset
from torchvision.transforms import ToTensor
import zipfile
import PIL

In [ ]:
# Load the pretrained ResNet34 model
resnet34 = models.resnet34(weights='IMAGENET1K_V1')

Downloading: "https://download.pytorch.org/models/resnet34-b627a593.pth" to /root/.cache/torch/hub/checkpoints/resnet34-b627a593.pth
100%|██████████| 83.3M/83.3M [00:00<00:00, 101MB/s]


In [ ]:

#load resnet to gpu
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
resnet34.to(device)

ResNet(
  (conv1): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
  (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (relu): ReLU(inplace=True)
  (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
  (layer1): Sequential(
    (0): BasicBlock(
      (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
      (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    )
    (1): BasicBlock(
      (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
  

In [ ]:
data_transforms = {
    'train': transforms.Compose([
        transforms.Resize((250,250)),
        transforms.RandomHorizontalFlip(),
        transforms.ToTensor(),
        transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
    ]),
    'val': transforms.Compose([
        transforms.Resize((250,250)),
        transforms.RandomHorizontalFlip(),
        transforms.ToTensor(),
        transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
    ]),
}

In [ ]:
class ZipDataset(Dataset):
    def __init__(self, root_path, var,trns, cache_into_memory=False):
        if cache_into_memory:
            f = open(root_path, 'rb')
            self.zip_content = f.read()
            f.close()
            self.zip_file = zipfile.ZipFile(io.BytesIO(self.zip_content), 'r')
        else:
            self.zip_file = zipfile.ZipFile(root_path, 'r')
        self.name_list = list(filter(lambda x: var in x and (x[-4:] == '.jpg'), self.zip_file.namelist()))
        species = list(filter(lambda x: x is not None,[filename.split('/')[2] if len(filename.split('/'))>3 else None for filename in self.name_list]))
        self.species_dict={name:iter for iter,name in enumerate(np.unique(species))}
        self.to_tensor = ToTensor()
        self.transf = trns[var]
    def __getitem__(self, key):
      buf = self.zip_file.read(name=self.name_list[key])
      img = cv2.imdecode(np.frombuffer(buf, dtype=np.uint8), cv2.IMREAD_COLOR)
      img = PIL.Image.fromarray(img)
      target = self.species_dict[self.name_list[key].split("/")[2]]
      img = self.transf(img)
      return img, target

    def __len__(self):
        return len(self.name_list)

In [ ]:
path1= '/content/drive/MyDrive/OG_best_dataset.zip'
ZipDataset(root_path=path1, var='train', trns=data_transforms).species_dict

NameError: name 'ZipDataset' is not defined

In [ ]:
path1= '/content/drive/MyDrive/OG_best_dataset.zip'
dataset_train = ZipDataset(root_path = path1,var='train', trns = data_transforms, cache_into_memory=False)
dataloader_train = torch.utils.data.DataLoader(dataset_train, batch_size=256, shuffle=True)
dataset_val = ZipDataset(path1,var='val',trns = data_transforms,cache_into_memory=False)
dataloader_val = torch.utils.data.DataLoader(dataset_val, batch_size=256, shuffle=True)
image_datasets = {'train': dataset_train, 'val': dataset_val}
dataloaders = {'train': dataloader_train, 'val': dataloader_val}
dataset_sizes = {'train': len(image_datasets['train']),'val': len(image_datasets['val'])}
print(dataset_sizes)

NameError: name 'ZipDataset' is not defined

In [ ]:
# def imshow(inp, title=None):
#     """Display image for Tensor."""
#     inp = inp.numpy().transpose((1, 2, 0))
#     mean = np.array([0.485, 0.456, 0.406])
#     std = np.array([0.229, 0.224, 0.225])
#     inp = std * inp + mean
#     inp = np.clip(inp, 0, 1)
#     plt.imshow(inp)
#     if title is not None:
#         plt.title(title)
#     plt.pause(0.001)  # pause a bit so that plots are updated


# # Get a batch of training data
# inputs, classes = next(iter(dataloaders['train']))
# print(classes)

# # Make a grid from batch
# out = torchvision.utils.make_grid(inputs)

# imshow(out, title=[dataset_train.species_dict[x] for x in classes])


In [ ]:
class_names = dataset_train.species_dict
print(class_names)
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")


{'Agaricus_augustus': 0, 'Agaricus_campestris': 1, 'Agaricus_xanthodermus': 2, 'Amanita_augusta': 3, 'Amanita_bisporigera': 4, 'Amanita_calyptroderma': 5, 'Amanita_citrina': 6, 'Amanita_flavoconia': 7, 'Amanita_flavorubens': 8, 'Amanita_fulva': 9, 'Amanita_jacksonii': 10, 'Amanita_junquillea': 11, 'Amanita_novinupta': 12, 'Amanita_pantherina': 13, 'Amanita_parcivolvata': 14, 'Amanita_persicina': 15, 'Amanita_phalloides': 16, 'Amanita_rubescens': 17, 'Amanita_vaginata': 18, 'Amanita_velosa': 19, 'Amanita_xanthocephala': 20, 'Aureoboletus_betula': 21, 'Aureoboletus_mirabilis': 22, 'Aureoboletus_russellii': 23, 'Baorangia_bicolor': 24, 'Boletus_edulis': 25, 'Boletus_reticulatus': 26, 'Boletus_rubriceps': 27, 'Cantharellus_californicus': 28, 'Cantharellus_cibarius': 29, 'Cantharellus_cinnabarinus': 30, 'Cantharellus_formosus': 31, 'Cantharellus_lateritius': 32, 'Cantharellus_minor': 33, 'Cerioporus_squamosus': 34, 'Chalciporus_piperatus': 35, 'Chlorophyllum_brunneum': 36, 'Chlorophyllum_mo

In [ ]:
# def imshow(inp, title=None):
#     """Display image for Tensor."""
#     inp = inp.numpy().transpose((1, 2, 0))
#     mean = np.array([0.485, 0.456, 0.406])
#     # std = np.array([0.229, 0.224, 0.225])
#     inp = std * inp + mean
#     inp = np.clip(inp, 0, 1)
#     plt.imshow(inp)
#     if title is not None:
#         plt.title(title)
#     plt.pause(0.001)  # pause a bit so that plots are updated


# # Get a batch of training data
# inputs, classes = next(iter(dataloaders['train']))

# # Make a grid from batch
# out = torchvision.utils.make_grid(inputs)

# imshow(out, title=[class_names[x] for x in classes])

In [ ]:

#Model conditions
new_model = resnet34
num_classes = 127  # Suposición
in_features = new_model.fc.in_features
new_model.fc = nn.Linear(in_features, num_classes)



In [ ]:
#setup optimizers
criterion = nn.CrossEntropyLoss()
optimizer = optim.SGD(new_model.parameters(), lr=0.001, momentum=0.9)
scheduler = lr_scheduler.MultiStepLR(optimizer,
                        milestones=[8, 24, 28], # List of epoch indices
                        gamma =0.5) # Multiplicative factor of learning rate decay

In [ ]:
def train_model(model, criterion, optimizer, scheduler, num_epochs=4):
    since = time.time()
    # Create a temporary directory to save training checkpoints
    with TemporaryDirectory() as tempdir:
        best_model_params_path = os.path.join(tempdir, 'best_model_params.pt')

        torch.save(model.state_dict(), best_model_params_path)
        best_acc = 0.0

        for epoch in range(num_epochs):
            print(f'Epoch {epoch}/{num_epochs - 1}')
            print('-' * 10)

            # Each epoch has a training and validation phase
            for phase in ['train', 'val']:
                if phase == 'train':
                    model.train()  # Set model to training mode
                else:
                    model.eval()   # Set model to evaluate mode

                running_loss = 0.0
                running_corrects = 0

                # Iterate over data.
                for inputs, labels in tqdm(dataloaders[phase],leave=False):
                    try:
                      inputs = inputs.to(device)
                      labels = labels.to(device)

                      # zero the parameter gradients
                      optimizer.zero_grad()

                      # forward
                      # track history if only in train
                      with torch.set_grad_enabled(phase == 'train'):
                          outputs = model(inputs)
                          _, preds = torch.max(outputs, 1)
                          loss = criterion(outputs, labels)

                          # backward + optimize only if in training phase
                          if phase == 'train':
                              loss.backward()
                              optimizer.step()
                    except Exception as e:
                      pass
                    # statistics
                    running_loss += loss.item() * inputs.size(0)
                    running_corrects += torch.sum(preds == labels.data)
                if phase == 'train':
                    scheduler.step()

                epoch_loss = running_loss / dataset_sizes[phase]
                epoch_acc = running_corrects.double() / dataset_sizes[phase]

                print(f'{phase} Loss: {epoch_loss:.4f} Acc: {epoch_acc:.4f}')

                # deep copy the model
                if phase == 'val' and epoch_acc > best_acc:
                    best_acc = epoch_acc
                    torch.save(model.state_dict(), best_model_params_path)

            print()

        time_elapsed = time.time() - since
        print(f'Training complete in {time_elapsed // 60:.0f}m {time_elapsed % 60:.0f}s')
        print(f'Best val Acc: {best_acc:4f}')

        # load best model weights

        model.load_state_dict(torch.load(best_model_params_path))
    return model

In [ ]:
new_model.load_state_dict(torch.load('/content/drive/MyDrive/saved_parameters/myshroomweights127final.pth'))


<All keys matched successfully>

In [ ]:

param_path='/content/drive/MyDrive/saved_parameters/muhsroomIDweights.pth'
new_model.to(device)
myshroom_model = train_model(new_model,criterion,optimizer,scheduler)
torch.save(myshroom_model.state_dict(),param_path )

NameError: name 'new_model' is not defined